In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math
from config import *

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'Price', 'FPL_pos', 'FPL_points',
       'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == NEXT_GAMEWEEK]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2084.863525,1738.765137,14.0,FWD,NaN,7.444841
51,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1954.190918,1835.312744,12.5,MID,NaN,6.869509
79,James Maddison,4.0,2023-24,Tottenham,Burnley,0.0,1836.362915,1713.381714,7.7,MID,NaN,6.609939
106,James Ward-Prowse,4.0,2023-24,West Ham,Luton,0.0,1803.122559,1597.144409,6.0,MID,NaN,6.375581
151,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1831.992188,1653.167114,6.8,MID,NaN,6.039028
204,Carlton Morris,4.0,2023-24,Luton,West Ham,1.0,1597.144409,1803.122559,5.5,FWD,NaN,5.811387
239,Axel Disasi,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1786.376709,1675.023193,5.1,DEF,NaN,5.710020
277,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2084.863525,1738.765137,7.6,MID,NaN,5.598909
309,Bruno Borges Fernandes,4.0,2023-24,Manchester Utd,Arsenal,0.0,1868.610107,1920.906372,8.5,MID,NaN,5.507563
311,Moussa Diaby,4.0,2023-24,Aston Villa,Liverpool,0.0,1835.312744,1954.190918,6.6,MID,NaN,5.504087


In [8]:
# df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((526, 12), (59, 12), (183, 12), (223, 12), (61, 12))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction
9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2084.863525,1738.765137,14.0,FWD,NaN,7.444841
51,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1954.190918,1835.312744,12.5,MID,NaN,6.869509
79,James Maddison,4.0,2023-24,Tottenham,Burnley,0.0,1836.362915,1713.381714,7.7,MID,NaN,6.609939
106,James Ward-Prowse,4.0,2023-24,West Ham,Luton,0.0,1803.122559,1597.144409,6.0,MID,NaN,6.375581
151,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1831.992188,1653.167114,6.8,MID,NaN,6.039028


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:
df_original = df.copy()

In [13]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [14]:
smallDB = []
for i, row in df.iterrows():
    smallDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [15]:
def find_most_expensive(playersDB):
    index = 0
    price = 0
    for i, p in enumerate(playersDB):
        if p.price > price:
            index = i
            price = p.price
    return index, price
    

In [16]:
def find_N_cheapest_on_position(playersDB, n_players, position):
    output = []
    for p in playersDB:
        if p.position == position:
            if len(output) < n_players:
                output.append(p)
            else:
                i, price = find_most_expensive(output)
                if p.price < price:
                    output[i] = p
    return output

In [17]:
find_N_cheapest_on_position(playersDB, 1, "FWD")[0].__str__()

'(Folarin Balogun, FWD, 4.4, 1.67, Arsenal) - (fixtures: 1.0)'

In [18]:
# for p in playersDB:
#     print(p.__str__())

In [19]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [20]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 2, 3],
    [5, 3, 2],
    [5, 4, 1],
]

In [21]:
budget = 100

In [22]:
def fill_bench(playersDB, formation):
    n_gk, n_def, n_mid, n_fwd = 1, 5 - formation[0], 5 - formation[1], 3 - formation[2]
    gks = find_N_cheapest_on_position(playersDB, n_gk, "GK")
    defs = find_N_cheapest_on_position(playersDB, n_def, "DEF")
    mids = find_N_cheapest_on_position(playersDB, n_mid, "MID")
    fwds = find_N_cheapest_on_position(playersDB, n_fwd, "FWD")
    print(formation, n_gk, n_def, n_mid, n_fwd)
    bench = gks + defs + mids + fwds
    for p in bench:
        print(p.__str__())
        p.benched = True
    bench_price = sum([p.price for p in bench])
    print(bench_price)
    return bench, bench_price

In [23]:
best_teams = []
best_team = None
best_team_score = 0
best_bench = None

for formation in possible_formations:
    bench, bench_price = fill_bench(playersDB, formation)
    best_team_for_formation = best_full_teams(smallDB, [formation], budget - bench_price)
    
    if best_team_for_formation[0][1] > best_team_score:
        best_team = best_team_for_formation
        best_team_score = best_team_for_formation[0][1]
        best_bench = bench
        
for p in best_team[0][2]:
    p.benched = True
for p in best_bench:
    best_team[0][2].append(p)
        
    

[3, 4, 3] 1 2 1 0
(Matt Turner, GK, 4.0, 0.28, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.31, Brighton) - (fixtures: 1.0)
(Auston Trusty, DEF, 4.0, 3.01, Sheffield United) - (fixtures: 1.0)
(Jakub Moder, MID, 4.4, 1.98, Brighton) - (fixtures: 1.0)
16.3
With formation [3, 4, 3]: 65.37  | (price = 81.3)
(Guglielmo Vicario, GK, 5.0, 5.49, Tottenham) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 5.71, Chelsea) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.6, 5.42, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 4.92, Liverpool) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.87, Liverpool) - (fixtures: 1.0)
(James Maddison, MID, 7.7, 6.61, Tottenham) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.0, 6.38, West Ham) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 6.04, Brentford) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.0, 7.44, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.81, Luton) - (fixtures: 1.0)
(Yoane Wissa, FWD, 6.2, 4.68, Brentford) - (fixtu

In [24]:
for p in best_team[0][2]:
    print(p.__str__())

(Guglielmo Vicario, GK, 5.0, 5.49, Tottenham) - (fixtures: 1.0)
(Axel Disasi, DEF, 5.1, 5.71, Chelsea) - (fixtures: 1.0)
(Destiny Udogie, DEF, 4.6, 5.42, Tottenham) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 7.9, 4.92, Liverpool) - (fixtures: 1.0)
(Mohamed Salah, MID, 12.5, 6.87, Liverpool) - (fixtures: 1.0)
(James Maddison, MID, 7.7, 6.61, Tottenham) - (fixtures: 1.0)
(James Ward-Prowse, MID, 6.0, 6.38, West Ham) - (fixtures: 1.0)
(Bryan Mbeumo, MID, 6.8, 6.04, Brentford) - (fixtures: 1.0)
(Moussa Diaby, MID, 6.6, 5.5, Aston Villa) - (fixtures: 1.0)
(Erling Haaland, FWD, 14.0, 7.44, Manchester City) - (fixtures: 1.0)
(Carlton Morris, FWD, 5.5, 5.81, Luton) - (fixtures: 1.0)
(Matt Turner, GK, 4.0, 0.28, Nott'ham Forest) - (fixtures: 1.0)
(Tariq Lamptey, DEF, 3.9, 2.31, Brighton) - (fixtures: 1.0)
(Auston Trusty, DEF, 4.0, 3.01, Sheffield United) - (fixtures: 1.0)
(Folarin Balogun, FWD, 4.4, 1.67, Arsenal) - (fixtures: 1.0)


In [25]:
%%time
# best_team = best_full_teams(smallDB, possible_formations, budget)

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
f"Best team score: {best_team[0][1]}"

'Best team score: 66.19'

In [27]:
best_team[0]

([3, 5, 2],
 66.19,
  <knapsack_problem.player.Player at 0x17ad9be4d90>])

In [28]:
def best_team_to_df(best_team_squad, df):
    names = [p.name for p in best_team_squad]
    print(names)
    custom_fpl_order = ["FWD", "MID", "DEF", "GK"]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    output = df[df["Name_original"].isin(names)].reset_index()
    output["First_eleven"] = True
    output.loc[11:15, "First_eleven"] = False
    output = output.sort_values(by=["First_eleven", "FPL_pos", "Price"], ascending=False)
    
    return output

In [29]:
best_squad = best_team_to_df(best_team[0][2], df_original)
best_squad

['Guglielmo Vicario', 'Axel Disasi', 'Destiny Udogie', 'Trent Alexander-Arnold', 'Mohamed Salah', 'James Maddison', 'James Ward-Prowse', 'Bryan Mbeumo', 'Moussa Diaby', 'Erling Haaland', 'Carlton Morris', 'Matt Turner', 'Tariq Lamptey', 'Auston Trusty', 'Folarin Balogun']


,index,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,Price,FPL_pos,FPL_points,Prediction,First_eleven
8,314,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1836.362915,1713.381714,5.0,GK,NaN,5.489847,True
10,576,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1954.190918,1835.312744,7.9,DEF,NaN,4.915789,True
6,239,Axel Disasi,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1786.376709,1675.023193,5.1,DEF,NaN,5.710020,True
9,338,Destiny Udogie,4.0,2023-24,Tottenham,Burnley,0.0,1836.362915,1713.381714,4.6,DEF,NaN,5.416049,True
1,51,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1954.190918,1835.312744,12.5,MID,NaN,6.869509,True
2,79,James Maddison,4.0,2023-24,Tottenham,Burnley,0.0,1836.362915,1713.381714,7.7,MID,NaN,6.609939,True
4,151,Bryan Mbeumo,4.0,2023-24,Brentford,Bournemouth,1.0,1831.992188,1653.167114,6.8,MID,NaN,6.039028,True
7,311,Moussa Diaby,4.0,2023-24,Aston Villa,Liverpool,0.0,1835.312744,1954.190918,6.6,MID,NaN,5.504087,True
3,106,James Ward-Prowse,4.0,2023-24,West Ham,Luton,0.0,1803.122559,1597.144409,6.0,MID,NaN,6.375581,True
0,9,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2084.863525,1738.765137,14.0,FWD,NaN,7.444841,True


In [30]:
best_squad.to_csv("predictions/best_squad.csv")